In [4]:
import pandas as pd
import numpy as np
from scripts.h2py import ignores


def common_timestamps(list_of_lists):
    """
    Trả về list các timestamp xuất hiện trong tất cả các list đầu vào.

    Args:
        list_of_lists: list các list, mỗi list chứa các timestamp

    Returns:
        list các timestamp xuất hiện trong tất cả các list, theo thứ tự sorted
    """
    if not list_of_lists:
        return []
    common_set = set(list_of_lists[0])  # Chuyển list đầu tiên thành set làm cơ sở
    for lst in list_of_lists[1:]:  # Lấy giao với các list tiếp theo
        common_set &= set(lst)  # intersection
    return sorted(common_set)


def create_tensor_form_csv(list_path):
    list_df = []
    for r in list_path:
        list_df.append(pd.read_csv(r))

    list_timestamp_series = [df['timestamp'] for df in list_df]

    timestamp = common_timestamps(list_timestamp_series)

    list_df = [df[df['timestamp'].isin(timestamp)] for df in list_df]

    # Lấy danh sách timestamp & variable theo thứ tự
    unique_vars = sorted(pd.concat([df["variable"] for df in list_df]).unique())
    unique_times = sorted(pd.concat([df["timestamp"] for df in list_df]).unique())

    # Map timestamp và variable về chỉ số
    time_to_index = {t: i for i, t in enumerate(unique_times)}
    var_to_index = {v: i for i, v in enumerate(unique_vars)}

    # Max row/col
    all_rows = pd.concat([df['row'] for df in list_df])
    all_cols = pd.concat([df['col'] for df in list_df])
    min_row, max_row = all_rows.min(), all_rows.max()
    min_col, max_col = all_cols.min(), all_cols.max()

    # Tạo array 4 chiều: time × variable × row × col
    a = np.zeros((len(unique_times),
                  len(unique_vars),
                  max_row - min_row + 1,
                  max_col - min_col + 1))

    # Đổ dữ liệu
    for df in list_df:
        for _, r in df.iterrows():
            t = time_to_index[r["timestamp"]]
            v = var_to_index[r["variable"]]
            a[t, v, r["row"] - min_row, r["col"] - min_col] = r["value"]

    return np.array(a), unique_times,unique_vars

In [6]:

# Tạo dữ liệu giả lập cho 2 DataFrame
hima_df = pd.DataFrame({
    'timestamp': ['20200101', '20200101', '20200102', '20200102'],
    'variable': ['var1', 'var2', 'var1', 'var2'],
    'row': [0, 0, 1, 1],
    'col': [0, 1, 0, 1],
    'value': [10, 20, 30, 40]
})

era5_df = pd.DataFrame({
    'timestamp': ['20200101', '20200102', '20200102', '20200103'],
    'variable': ['var1', 'var1', 'var2', 'var2'],
    'row': [0, 1, 1, 2],
    'col': [0, 0, 1, 1],
    'value': [100, 200, 300, 400]
})

hima_df.to_csv('../csv_data/test_hima.csv',index=False)
era5_df.to_csv('../csv_data/test_era5.csv', index=False)



In [8]:
paths = ['../csv_data/test_hima.csv' ,'../csv_data/test_era5.csv' ]
tensor, times, vars_ = create_tensor_form_csv(paths)

print("Tensor shape:", tensor.shape)
print("Timestamps:", times)
print("Variables:", vars_)
print("Tensor contents:\n", tensor)


Tensor shape: (2, 2, 2, 2)
Timestamps: [np.int64(20200101), np.int64(20200102)]
Variables: ['var1', 'var2']
Tensor contents:
 [[[[100.   0.]
   [  0.   0.]]

  [[  0.  20.]
   [  0.   0.]]]


 [[[  0.   0.]
   [200.   0.]]

  [[  0.   0.]
   [  0. 300.]]]]


In [6]:
import numpy as np
arr = np.load("../csv_data/y_hatinh.npy")
print(type(arr))
print(arr.shape)
print(arr.dtype)

<class 'numpy.ndarray'>
(2337, 21, 34)
float64


In [7]:
arr1 = np.load("../csv_data/x_hatinh.npy")
print(type(arr1))
print(arr1.shape)
print(arr1.dtype)

<class 'numpy.ndarray'>
(2337, 34, 21, 34)
float64


In [22]:

print((arr[:,0] == arr[:,1]).all())
# print('------------------------------------')
# print(arr[101][1])

False


In [14]:
print(arr[1][1])


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [10]:
import numpy as np

# Giả sử bạn đã load 2 mảng vào biến y và x
# y = np.load("./csv_data/y_hatinh.npy")
# x = np.load("./csv_data/x_hatinh.npy")

# --- Code kiểm tra các chiều ---

# 1. Kiểm tra chiều TIME (Thời gian): Cả hai mảng đều ở vị trí đầu tiên (index 0)
# y[0] vs x[0]
time_match = arr.shape[0] == arr1.shape[0]

# 2. Kiểm tra chiều ROW (Hàng):
# Row nằm ở vị trí thứ 2 trong y (index 1) và vị trí thứ 3 trong x (index 2)
# y[1] vs x[2]
row_match = arr.shape[1] == arr1.shape[2]

# 3. Kiểm tra chiều COL (Cột):
# Col nằm ở vị trí thứ 3 trong y (index 2) và vị trí thứ 4 trong x (index 3)
# y[2] vs x[3]
col_match = arr.shape[2] == arr1.shape[3]

# --- In kết quả ---

print(f"Kích thước mảng y (y.shape): {arr.shape}")
print(f"Kích thước mảng x (x.shape): {arr1.shape}")
print("-" * 30)

print(f"Kiểm tra chiều TIME (y[0] == x[0]): {time_match}. Giá trị: {arr.shape[0]} == {arr1.shape[0]}")
print(f"Kiểm tra chiều ROW (y[1] == x[2]): {row_match}. Giá trị: {arr.shape[1]} == {arr1.shape[2]}")
print(f"Kiểm tra chiều COL (y[2] == x[3]): {col_match}. Giá trị: {arr.shape[2]} == {arr1.shape[3]}")
print("-" * 30)

# Kiểm tra tổng thể
all_dimensions_match = time_match and row_match and col_match
print(f"Tất cả các chiều TIME, ROW, COL có khớp nhau: {all_dimensions_match}")

Kích thước mảng y (y.shape): (2337, 21, 34)
Kích thước mảng x (x.shape): (2337, 34, 21, 34)
------------------------------
Kiểm tra chiều TIME (y[0] == x[0]): True. Giá trị: 2337 == 2337
Kiểm tra chiều ROW (y[1] == x[2]): True. Giá trị: 21 == 21
Kiểm tra chiều COL (y[2] == x[3]): True. Giá trị: 34 == 34
------------------------------
Tất cả các chiều TIME, ROW, COL có khớp nhau: True


In [11]:
print(arr.shape[0])
print(arr1.shape[0])

2337
2337


In [12]:
# Giả định y=arr và x=arr1
def check_data_synchronization(y_arr, x_arr):
    # Lấy một lát cắt thời gian ngẫu nhiên (hoặc cố định, ví dụ index 10)
    t_index = 10

    # 1. So sánh lát cắt thời gian
    # Lấy lát cắt không gian (row, col) của y tại t_index
    y_spatial_slice = y_arr[t_index, :, :]  # Shape (21, 34)

    # Lấy lát cắt không gian (row, col) của x tại t_index (bỏ qua band)
    # Ta phải chọn một band bất kỳ (ví dụ band 0) để có shape (21, 34) để so sánh
    x_spatial_slice = x_arr[t_index, 0, :, :] # Shape (21, 34)

    # Kích thước phải khớp nhau (đã kiểm tra ở bước A)
    if y_spatial_slice.shape != x_spatial_slice.shape:
        return "LỖI SHAPE", False

    # 2. Kiểm tra tính nhất quán dữ liệu
    # Dùng numpy.array_equal để kiểm tra xem lát cắt không gian này có giống nhau không.
    # LƯU Ý QUAN TRỌNG: Hai mảng này KHÔNG BAO GIỜ được giống hệt nhau về giá trị
    # vì chúng là dữ liệu khác nhau (y là nhãn, x là đầu vào).
    # Ta phải kiểm tra xem chúng có được tạo ra từ cùng một LỆNH RESHAPE hay không.

    print(f"Kiểm tra tại Time Index {t_index}:")
    print(f"y-slice shape: {y_spatial_slice.shape}")
    print(f"x-band-0-slice shape: {x_spatial_slice.shape}")

    # CHỌN MỘT ĐIỂM NGẪU NHIÊN để xác nhận:
    # Ví dụ: kiểm tra giá trị tại (row 5, col 10)
    test_y_value = y_spatial_slice[5, 10]
    test_x_values = x_arr[t_index, :, 5, 10] # Lấy tất cả các band tại điểm đó

    # Ta không thể so sánh giá trị, mà phải so sánh LỌC.
    # Nếu dữ liệu của bạn có các giá trị NaN/0/None đặc trưng cho các điểm không hợp lệ
    # thì việc kiểm tra các giá trị đó có cùng vị trí sẽ xác nhận sự đồng bộ.

    # Ví dụ kiểm tra các điểm bị thiếu dữ liệu (ví dụ: giá trị = 0)
    y_zeros = np.sum(y_spatial_slice == 0)
    x_zeros_band0 = np.sum(x_spatial_slice == 0)

    if y_zeros == x_zeros_band0:
        return f"OK. Số lượng điểm '0' (hoặc NaN) trùng nhau ({y_zeros}) tại index {t_index}. Có vẻ đồng bộ.", True
    else:
        return f"CẢNH BÁO. Số lượng điểm '0' KHÔNG trùng nhau tại index {t_index} ({y_zeros} vs {x_zeros_band0}). KHÔNG ĐỒNG BỘ.", False

# Thực thi kiểm tra (Giả sử arr và arr1 đã được load)
result_message, is_synchronized = check_data_synchronization(arr, arr1)
print("\n--- KIỂM TRA ĐỒNG BỘ DỮ LIỆU ---")
print(result_message)

Kiểm tra tại Time Index 10:
y-slice shape: (21, 34)
x-band-0-slice shape: (21, 34)

--- KIỂM TRA ĐỒNG BỘ DỮ LIỆU ---
CẢNH BÁO. Số lượng điểm '0' KHÔNG trùng nhau tại index 10 (519 vs 393). KHÔNG ĐỒNG BỘ.
